In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz

# Load the column variations and their corresponding standard names from a single CSV file
columns_df = pd.read_csv('dataset-clean.csv')

# Extract column variations and standard names into separate lists
column_variations = columns_df['SOURCE_FIELD_NAME'].tolist()
standard_names = columns_df['FIELD_NAME'].tolist()

# Define new column name variations for prediction
new_column_names = [
    'ACNO', 'BIRTH',
    'APPL_CODE12', 'APPLIC_TRID51', 'APPLIC', 'APPL', 'APPTRID',
    'CUST_TYPE', 'CUSTYPE', 'CTYPE'
]

# Function to predict the standard column name using fuzzy matching
def predict_column_name(new_name, column_variations, standard_names):
    # Use fuzzy matching to compare the new name with all column variations
    similarities = [(var, fuzz.ratio(new_name, var)) for var in column_variations]
    # Sort by similarity score in descending order
    similarities.sort(key=lambda x: x[1], reverse=True)

    # Check for threshold similarity (to avoid incorrect matches)
    threshold = 65  # Lowered threshold for better matching
    if similarities[0][1] < threshold:
        return "Unknown"  # If the match is not strong enough, return "Unknown"

    # Return the standard name corresponding to the highest similarity match
    max_similarity_idx = column_variations.index(similarities[0][0])
    return standard_names[max_similarity_idx]

# Function to predict the SQL data type based on the column name
def predict_data_type(column_name):
    # Map column names to SQL data types
    column_name = column_name.lower()

    if 'account' in column_name or 'phn' in column_name or 'number' in column_name:
        return 'VARCHAR(255)'  # Account Number, Phone Number, etc. are typically VARCHAR
    elif 'date' in column_name or 'dob' in column_name:
        return 'DATE'  # Date fields like Date of Birth are DATE
    elif 'address' in column_name or 'street' in column_name:
        return 'VARCHAR(255)'  # Address fields are VARCHAR
    else:
        return 'TEXT'  # Default to TEXT if no specific pattern is found

# Predict standard column names and their corresponding data types for the new variations
predicted_column_names_and_types = []

predicted_result = []

for new_column in new_column_names:
    predicted_name = predict_column_name(new_column, column_variations, standard_names)

    # If "Unknown" is returned, handle this case.
    if predicted_name == "Unknown":
        print(f"Warning: Unable to predict a standard name for column '{new_column}'.")
        predicted_type = 'TEXT'  # Default to TEXT type if not recognized
    else:
        predicted_type = predict_data_type(predicted_name)

    predicted_result.append((new_column, predicted_name))
    predicted_column_names_and_types.append((predicted_name, predicted_type))


# SQL CREATE TABLE generation based on predicted column names and types
def generate_create_table_query(predicted_column_names_and_types, table_name):
    sql_query = f"CREATE TABLE {table_name} (\n"

    for col_name, col_type in predicted_column_names_and_types:
        # Convert column name to lowercase with underscores (standard SQL style)
        sql_query += f"    {col_name.replace(' ', '_').lower()} {col_type} NOT NULL,\n"

    # Remove the trailing comma for the last column
    sql_query = sql_query.rstrip(',\n') + "\n);"

    return sql_query

# Generate the SQL CREATE TABLE query based on the predicted column names and types
table_name = "predicted_table"
create_table_query = generate_create_table_query(predicted_column_names_and_types, table_name)

# Output the predicted column names, data types, and the SQL query
# print("Predicted Column Names and Types:")
# for col_name, col_type in predicted_column_names_and_types:
#     print(f"{col_name}: {col_type}")


# Print the predicted FIELD_NAME for each SOURCE_FIELD_NAME
for source, predicted_field in predicted_result:
    print(f"Predicted FIELD_NAME for '{source}': {predicted_field}")

print("\nGenerated SQL CREATE TABLE Query:")
print(create_table_query)


Predicted FIELD_NAME for 'ACNO': CUST_NO
Predicted FIELD_NAME for 'BIRTH': CUST_CREATE_DT
Predicted FIELD_NAME for 'APPL_CODE12': APPL_CD
Predicted FIELD_NAME for 'APPLIC_TRID51': TRAILER_ID
Predicted FIELD_NAME for 'APPLIC': APPL_CD
Predicted FIELD_NAME for 'APPL': BK_CD
Predicted FIELD_NAME for 'APPTRID': TRAILER_ID
Predicted FIELD_NAME for 'CUST_TYPE': ID_SUB_CTG
Predicted FIELD_NAME for 'CUSTYPE': CUST_TP
Predicted FIELD_NAME for 'CTYPE': CUST_TP

Generated SQL CREATE TABLE Query:
CREATE TABLE predicted_table (
    cust_no TEXT NOT NULL,
    cust_create_dt TEXT NOT NULL,
    appl_cd TEXT NOT NULL,
    trailer_id TEXT NOT NULL,
    appl_cd TEXT NOT NULL,
    bk_cd TEXT NOT NULL,
    trailer_id TEXT NOT NULL,
    id_sub_ctg TEXT NOT NULL,
    cust_tp TEXT NOT NULL,
    cust_tp TEXT NOT NULL
);
